#### Problem:

Create a 100x100 grid with obstacles in between 2 random points. 
Build an MDP based RL agent to optimise both policies and actions at every state.
Benchmark DP method with other RL solutions for the same problem.

#### Algorithm:

Define the Grid and Obstacles: Create a 100x100 grid with randomly placed obstacles.

Define MDP (Markov Decision Process): Define states, actions, rewards, and transitions for this environment.

Dynamic Programming Solution: Implement a Dynamic Programming approach for policy optimization.

Q-Learning Agent: Implement a Q-learning agent as a comparison for the DP solution.

Benchmark and Comparison: Evaluate the agents' performances.

#### Implementation:

In [13]:
import numpy as np
import random
from collections import defaultdict

In [14]:
GRID_SIZE = 100
OBSTACLE_PROBABILITY = 0.2

In [15]:
ACTIONS = [(0, 1), (0, -1), (1, 0), (-1, 0)]
ACTION_MAP = {0: (0, 1), 1: (0, -1), 2: (1, 0), 3: (-1, 0)}

###### Dynamic Programming Agent

In [16]:
class GridMDP:
    def __init__(self, grid_size, obstacle_prob):
        self.grid_size = grid_size
        self.obstacle_prob = obstacle_prob
        self.grid = self.create_grid()
        self.start, self.goal = self.get_start_goal()

    def create_grid(self):
        grid = np.zeros((self.grid_size, self.grid_size), dtype=int)
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if random.random() < self.obstacle_prob:
                    grid[i, j] = -1  #obstacle
        return grid

    def get_start_goal(self):
        while True:
            start = (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))
            goal = (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))
            if start != goal and self.grid[start] == 0 and self.grid[goal] == 0:
                return start, goal

    def is_valid(self, pos):
        x, y = pos
        return 0 <= x < self.grid_size and 0 <= y < self.grid_size and self.grid[x, y] == 0

    def step(self, state, action):
        next_state = (state[0] + action[0], state[1] + action[1])
        if not self.is_valid(next_state):
            return state, -1  #Penalty 
        if next_state == self.goal:
            return next_state, 100  #reward 
        return next_state, -1  #step cost

###### Q-Learning Agent

In [17]:
class QLearningAgent:
    def __init__(self, env, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=5000):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.episodes = episodes
        self.q_table = defaultdict(lambda: np.zeros(len(ACTIONS)))

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(ACTIONS)
        state_index = tuple(state)
        return ACTIONS[np.argmax(self.q_table[state_index])]

    def learn(self):
        for episode in range(self.episodes):
            state = self.env.start
            while state != self.env.goal:
                action = self.choose_action(state)
                next_state, reward = self.env.step(state, action)
                state_index = tuple(state)
                next_index = tuple(next_state)
                best_next_action = np.max(self.q_table[next_index])
                self.q_table[state_index][ACTIONS.index(action)] += self.alpha * (
                    reward + self.gamma * best_next_action - self.q_table[state_index][ACTIONS.index(action)]
                )
                state = next_state

In [18]:
# Benchmarking
env = GridMDP(GRID_SIZE, OBSTACLE_PROBABILITY)

In [19]:
# Dynamic programming agent
dp_agent = DPAgent(env)
dp_agent.policy_iteration()

In [20]:
# Q-Learning agent
q_agent = QLearningAgent(env)
q_agent.learn()

##### Comparing policies

In [22]:
def evaluate_agent(agent, episodes=100):
    success_count = 0
    total_steps = 0
    for _ in range(episodes):
        state = env.start
        steps = 0
        while state != env.goal:
            action = ACTIONS[agent.policy[state]] if isinstance(agent, DPAgent) else agent.choose_action(state)
            next_state, _ = env.step(state, action)
            state = next_state
            steps += 1
            if steps > 10000:  
                break
        if state == env.goal:
            success_count += 1
        total_steps += steps
    success_rate = success_count / episodes
    avg_steps = total_steps / episodes
    return success_rate, avg_steps

dp_success_rate, dp_avg_steps = evaluate_agent(dp_agent)
q_success_rate, q_avg_steps = evaluate_agent(q_agent)

print(f"DP Success Rate: {dp_success_rate}, DP Avg Steps: {dp_avg_steps}")
print(f"Q-Learning Success Rate: {q_success_rate}, Q-Learning Avg Steps: {q_avg_steps}")

DP Success Rate: 1.0, DP Avg Steps: 22.0
Q-Learning Success Rate: 1.0, Q-Learning Avg Steps: 25.08
